## Processo para utlização da biblioteca Selenium para webscraping no site Quinto Andar

# Nessa parte vamos importar bibliotecas necessárias para o codigo

Pode se perceber que não utlizamos todas nesse script, porem foi deixado caso querem usar outras estrategias para o script

In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import re
import time
import pandas as pd
import numpy as np
import requests
from time import sleep
from tqdm import tqdm

# Buscando os dados na web

AQUI IREMOS SALVAR TODOS OS NOMES DE CLASSES HTML PARA BUSCA  
Cada classe foi buscada no site quinto andar, fazendo a analise html do site

Preço do apartamento = CozyTypography xih2fc EKXjIf EqjlRj  
IPTU + Cond = CozyTypography xih2fc _72Hu5c Ci-jp3  
caracteristicas = CozyTypography o6Ojuq xih2fc EKXjIf A68t3o  
tipo  = Cozy__CardTitle-Metadata Dg2zLY  
endereço = CozyTypography xih2fc _72Hu5c Ci-jp3  




Vamos configurar o nosso browser para a abertura do site

In [4]:
driver = webdriver.Chrome()
driver.get('https://www.quintoandar.com.br/comprar/imovel/belo-horizonte-mg-brasil')

Iterando sobre o Botão ver mais para carregar todo elemento html do site.  

Lembrando que caso esse passo não seja feito vamos pegar apenas os empreendimentos carregados no html que ja esta carregado de inicio  

Caso seja utlizada a Biblioteca BeautifulSoup é necessario interagir com o botão ver mais utlizando a selenium, alem de passar a pagina para  
html e utilizar o metodo find_all ou selec ao inves de find_element  

A parte do range deve ser alterada para conseguir pegar todos os elementos, precisa ser trocado de acordo com o volume de apartamentos na região.  

No sleep deve ser alterado de forma que de tempo do seu computador carrega toda a pagina para executar o clique no botão novamente  

Aqui também utilizamos a biblioteca tqdm para saber qaunto do processo ja foi executado

In [5]:
sleep(30)
for i in tqdm(range(80), desc="Processando", unit=" clique"):
    botao = driver.find_element(By.CLASS_NAME, 'MuiBox-root.mui-d8ybqx')
    botao.click()
    sleep(10)

Processando: 100%|██████████| 80/80 [19:40<00:00, 14.76s/ clique]


Usando as classes escolhidas no html, lembrando que os espaços precisam ser substuitos por pontos no Selenium.  

Caso use BeautifulSoup não é necessario a substituição

In [6]:
emp_valor = driver.find_elements(By.CLASS_NAME, 'CozyTypography.xih2fc.EKXjIf.EqjlRj')
emp_iptu_cond = driver.find_elements(By.CLASS_NAME, 'Cozy__CardTitle-Subtitle.JyjznE')
emp_caracter = driver.find_elements(By.CLASS_NAME, 'CozyTypography.o6Ojuq.xih2fc.EKXjIf.A68t3o')
emp_tipo = driver.find_elements(By.CLASS_NAME, 'Cozy__CardTitle-Metadata.Dg2zLY')
emp_end = driver.find_elements(By.CSS_SELECTOR, 'h2.CozyTypography.xih2fc._72Hu5c.Ci-jp3')

Agora criei uma lista para cada variavel onde irei armazenar os valores das colunas para aplicar a um DataFrame

In [7]:
valores = [valor.text for valor in emp_valor]
iptu_condicoes = [iptu.text for iptu in emp_iptu_cond]
caracteristicas = [caracter.text for caracter in emp_caracter]
tipos = [tipo.text for tipo in emp_tipo]
enderecos = [end.text for end in emp_end]

Aqui escolhi criar um Dicionario para para amazenar os valores e os nomes das colunas do Dataframe

In [8]:
data_dict = {
    'Valor': valores,
    'IPTU_Condicao': iptu_condicoes,
    'Caracteristicas': caracteristicas,
    'Tipo': tipos,
    'Endereco': enderecos
}

In [64]:
df = pd.DataFrame(data_dict)

In [65]:
df.head(15)

,Valor,IPTU_Condicao,Caracteristicas,Tipo,Endereco
0,R$ 175.000,R$ 0 Condo. + IPTU,120 m² · 3 quartos,Casa,"Rua Dona Luiza, Milionários · Belo Horizonte"
1,R$ 250.000,R$ 0 Condo. + IPTU,150 m² · 3 quartos · 3 vagas,Casa,"R. Josefina Guimaraes Vieira, Jardim Felicidad..."
2,R$ 700.000,R$ 577 Condo. + IPTU,150 m² · 3 quartos · 2 vagas,Apartamento,"Rua Luiza Carvalho Torres, Santo Antônio · Bel..."
3,R$ 280.000,R$ 83 Condo. + IPTU,180 m² · 2 quartos · 3 vagas,Casa,"Rua Ponta Grossa, Milionários · Belo Horizonte"
4,R$ 255.000,R$ 100 Condo. + IPTU,120 m² · 2 quartos · 1 vaga,Casa em condomínio,"Rua Três Marias, Miramar (barreiro) · Belo Hor..."
5,R$ 180.000,R$ 35 Condo. + IPTU,73 m² · 3 quartos · 1 vaga,Apartamento,"Rua das Goiabeiras, Manacás · Belo Horizonte"
6,R$ 740.000,R$ 550 Condo. + IPTU,144 m² · 3 quartos · 1 vaga,Apartamento,"Estrada do Cercadinho, Jardim América · Belo H..."
7,R$ 250.000,R$ 0 Condo. + IPTU,120 m² · 2 quartos · 1 vaga,Casa,"Rua Três Marias, Miramar (barreiro) · Belo Hor..."
8,R$ 515.000,R$ 436 Condo. + IPTU,123 m² · 3 quartos · 2 vagas,Apartamento,"Rua Sagres, Prado · Belo Horizonte"
9,R$ 280.000,R$ 290 Condo. + IPTU,93 m² · 3 quartos · 1 vaga,Apartamento,"Rua Bueno do Prado, João Pinheiro · Belo Horiz..."


In [43]:
df.to_csv('teste.csv')

In [52]:
def safe_split_iptu(x):
    try:
        return x.split('+')[1]
    except IndexError:
        return None

def safe_split_condominio(x):
    try:
        return x.split('+')[0]
    except IndexError:
        return None

# Aplicando as funções e gerando valores
df['Iptu'] = df['IPTU_Condicao'].astype(str).apply(safe_split_iptu)
df['Condominio'] = df['IPTU_Condicao'].astype(str).apply(safe_split_condominio)

# Removendo as linhas com erro (valores None)
df = df.dropna(subset=['Iptu', 'Condominio'])


# Tratamento dos dados

Nessa primeira parte vamos tratar os dados e deixa-los quase 100% prontos para analises, separando colunas com mais de uma informação e  
retirando caracteres especiais

In [53]:
df['Valor'] = df['Valor'].astype(str).apply(lambda x : x.split(' ')[1])
#df['Iptu'] = df['IPTU_Condicao'].astype(str).apply(lambda x: x.split('+')[1])
#df['Condominio'] = df['IPTU_Condicao'].astype(str).apply(lambda x: x.split('+')[0])
df['Endereco'] = df['Endereco'].astype(str).apply(lambda x : x.split(',')[1].split('·')[0])
df['Condominio'] = df['Condominio'].astype(str).apply(lambda x: x.split(' ')[1])
df['Caracteristicas'] = df['Caracteristicas'].astype(str).apply(lambda x:x.split(' · '))

In [54]:
for i,r in df.iterrows():
    if len(r['Caracteristicas']) == 2:
        df.loc[i,'m2']=r['Caracteristicas'][0].split(' ')[0]
        df.loc[i,'quartos']=r['Caracteristicas'][1].split(' ')[0]
    else:
        df.loc[i,'m2']=r['Caracteristicas'][0].split(' ')[0]
        df.loc[i,'quartos']=r['Caracteristicas'][1].split(' ')[0]
        df.loc[i,'vagas'] = r['Caracteristicas'][2].split(' ')[0]

In [55]:
df['vagas'].fillna(0,inplace=True)

In [56]:
df = df.drop(['IPTU_Condicao','Caracteristicas'], axis=1)

Nessa segunda parte deixar os dados 100% prontos vamos deixar as colunas nos tipos corretos para analise

In [59]:
df.describe()

,Valor,Tipo,Endereco,Iptu,Condominio,m2,quartos,vagas
count,967,967,967,967,967,967,967,967
unique,209,4,174,1,494,139,6,9
top,450.000,Apartamento,Buritis,IPTU,0,60,3,1
freq,31,863,45,967,37,55,487,492


In [60]:
colunas_para_converter = ['Valor', 'Iptu', 'Condominio', 'm2', 'quartos', 'vagas']

for coluna in colunas_para_converter:
    df[coluna] = pd.to_numeric(df[coluna].str.replace('.', '').str.strip(), errors='coerce').fillna(0).astype(int)


C:\Users\henrique.figueiredo\AppData\Local\Temp\ipykernel_18396\3933995320.py:4: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df[coluna] = pd.to_numeric(df[coluna].str.replace('.', '').str.strip(), errors='coerce').fillna(0).astype(int)


In [61]:
df.head()

,Valor,Tipo,Endereco,Iptu,Condominio,m2,quartos,vagas
0,175000,Casa,Milionários,0,0,120,3,0
1,250000,Casa,Jardim Felicidade,0,0,150,3,3
2,700000,Apartamento,Santo Antônio,0,577,150,3,2
3,280000,Casa,Milionários,0,83,180,2,3
4,255000,Casa em condomínio,Miramar (barreiro),0,100,120,2,1


In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 972 entries, 0 to 971
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Valor            972 non-null    object
 1   IPTU_Condicao    972 non-null    object
 2   Caracteristicas  972 non-null    object
 3   Tipo             972 non-null    object
 4   Endereco         972 non-null    object
dtypes: object(5)
memory usage: 38.1+ KB


In [62]:
df.to_csv('base.csv')

In [63]:
df.head(15)

,Valor,Tipo,Endereco,Iptu,Condominio,m2,quartos,vagas
0,175000,Casa,Milionários,0,0,120,3,0
1,250000,Casa,Jardim Felicidade,0,0,150,3,3
2,700000,Apartamento,Santo Antônio,0,577,150,3,2
3,280000,Casa,Milionários,0,83,180,2,3
4,255000,Casa em condomínio,Miramar (barreiro),0,100,120,2,1
5,180000,Apartamento,Manacás,0,35,73,3,1
6,740000,Apartamento,Jardim América,0,550,144,3,1
7,250000,Casa,Miramar (barreiro),0,0,120,2,1
8,515000,Apartamento,Prado,0,436,123,3,2
9,280000,Apartamento,João Pinheiro,0,290,93,3,1
